We aim to analyse the linkedin jobs dataset to understand:

1. The biggest emplpoyers in the data/analyst space
2. The most in demand experience level in data/analyst space, broken down by states in the US.
3. Most popular employers and job titles
4. Most popular skills across all job postings
5. Which skills are most indemand in which industry
5. Most popular benefits across all job postings



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

file = ('/content/drive/MyDrive/Hackathon/postings2.csv')
df = pd.read_csv(file)
df.head(5)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,Experience
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Mid
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Mid
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Management
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,NaN,NaN,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Senior
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,NaN,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Mid


Creating dataframe of columns we want to work with from the main dataframe. We realised that quite a few of the fields that we wanted to work with,
like salary, were missing too many fields to be useful. We then created a another filtered dataframe to only include jobs that were either data related or analytics related.

In [4]:

postings_df = df[['job_id', 'company_name', 'title', 'description', 'location', 'formatted_experience_level', 'Experience', 'work_type', 'compensation_type', 'listed_time', 'expiry']]


In [5]:
pattern = '|'.join(['Business Analyst', 'Data Scientist', 'Business Intelligence Analyst', 'Data Engineer', 'Quantitative Analyst', 'Financial Analyst', 'Marketing Analyst', 'Financial Analyst', 'Marketing Analyst', 'Insights Analyst', 'Research Analyst', 'Data Consultant', 'Reporting Analyst', 'Analytics Consultant', 'Decision Support Analyst', 'Data Specialist', 'Analyst'])

filter_job = postings_df['title'].str.contains(pattern, case=False, na=False)

filtered_postings_df = postings_df[filter_job]
filtered_postings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5598 entries, 63 to 123771
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   job_id                      5598 non-null   int64  
 1   company_name                5555 non-null   object 
 2   title                       5598 non-null   object 
 3   description                 5598 non-null   object 
 4   location                    5598 non-null   object 
 5   formatted_experience_level  4273 non-null   object 
 6   Experience                  5598 non-null   object 
 7   work_type                   5598 non-null   object 
 8   compensation_type           1913 non-null   object 
 9   listed_time                 5598 non-null   float64
 10  expiry                      5598 non-null   float64
dtypes: float64(2), int64(1), object(8)
memory usage: 524.8+ KB


In [6]:
filtered_postings_df.duplicated().sum()

0

Checking for any NA fields. Compensation type was not useful to us but formatted_experience level was.
We went through a few iterations of trying to populate the formatted_experience column and what worked finally is a python script
to extract the experience level from the job title. So,for example, if the job title is Sr data analyst, the experience level became Senior.
After that script was run, we had an experience column and could drop the formatted_expeience_level column.

In [7]:

filtered_postings_df.isna().sum()

job_id                           0
company_name                    43
title                            0
description                      0
location                         0
formatted_experience_level    1325
Experience                       0
work_type                        0
compensation_type             3685
listed_time                      0
expiry                           0
dtype: int64

In [8]:
filtered_postings_df=filtered_postings_df.drop(['formatted_experience_level'], axis=1)
filtered_postings_df['company_name'].dropna(inplace=True)
#filtered_postings_df.head(5)

In [9]:
n = len(pd.unique(filtered_postings_df['title']))
print(n)

3488


We needed to clean some of the columns, like time and location, into more usable formats.
After splitting location into city and state, there were many fields that didn't follow any standard, for example the job would be for the Texas Metro area instead of just saying Texas. We made a keyword dictionary to remap the irregular entries to a state abbreviation, so Texas becomes TX.

In [10]:
#Converting the time column from epoch to standard time
filtered_postings_df['listed_time'] = pd.to_datetime(filtered_postings_df['listed_time']/1000, unit='s')
filtered_postings_df['expiry'] = pd.to_datetime(filtered_postings_df['expiry']/1000, unit='s')
# filtered_postings_df.head(5)

In [11]:
filtered_postings_df['expiry_month'] = filtered_postings_df['expiry'].dt.month

#filtered_postings_df.head(5)

In [12]:
expiry_month = filtered_postings_df.groupby(['expiry_month']).size()

In [13]:
#Splitting up the Location column into city and state columns so that we could analyse jobs based on locations.
filtered_postings_df['City']= df['location'].str.split(',', expand=True)[0]
filtered_postings_df['State']= df['location'].str.split(',', expand=True)[1]
filtered_postings_df.head(5)

,job_id,company_name,title,description,location,Experience,work_type,compensation_type,listed_time,expiry,expiry_month,City,State
63,1880409118,Kona Medical Consulting,Board Certified Behavior Analyst,JOB OVERVIEW:\nThe BCBA will provide support t...,"McDonough, GA",Mid,FULL_TIME,NaN,2024-04-19 06:42:23,2024-05-19 06:42:23,5,McDonough,GA
69,2147609785,The Job Network,FP&A Analyst,Integrity is one of the nation’s leading indep...,"Dallas, TX",Mid,FULL_TIME,NaN,2024-04-12 06:46:24,2024-05-12 06:46:24,5,Dallas,TX
179,3582587683,McNaughton Bros. Inc.,Accounts Analyst,Company DescriptionMcNaughton Bros. Inc. is a ...,"Indiana, PA",Mid,FULL_TIME,BASE_SALARY,2024-04-11 19:11:08,2024-05-11 19:11:08,5,Indiana,PA
283,3742692445,ZenithMinds Inc,Sr Data Engineer with Kafka,Data Engineer with Kafka (W2 Only)💯% Remote\nM...,"Austin, TX",Senior,FULL_TIME,NaN,2024-04-15 19:17:53,2024-10-12 19:17:15,10,Austin,TX
301,3763494664,HAPPI Health,Help Desk Analyst,Job DescriptionAssures the operation of the El...,Huntsville-Decatur-Albertville Area,Mid,FULL_TIME,NaN,2024-04-15 20:13:13,2024-05-15 20:13:13,5,Huntsville-Decatur-Albertville Area,None


In [14]:

specific_areas_abbrev={
    'United States': 'US',
    'Nebraska Metropolitan Area': 'NE',
    'Ohio Metropolitan Area': 'OH',
    'South Carolina Metropolitan Area': 'SC',
    'Texas Metropolitan Area': 'TX',
    'Oregon Metropolitan Area': 'OR',
    'Illinois Metropolitan Area': 'IL',
    'South Carolina Area': 'SC',
    'Massachusetts Metropolitan Area': 'MA',
    'Wisconsin Metropolitan Area': 'WI',
    'New York Metropolitan Area': 'NY',
    'North Carolina Metropolitan Area': 'NC',
    'Louisiana Metropolitan Area': 'LA',
    'Virginia Metropolitan Area': 'VA',
    'Georgia Area': 'GA',
    'South Holland': 'IL',
    'Indiana Metropolitan Area': 'IN',
    'Missouri Area': 'MO',
    'Illinois Area': 'IL',
    'North Carolina Area': 'NC',
    'IL Area': 'IL',
    'Maine Metropolitan Area': 'ME',
    'Minnesota Area': 'MN',
    'Kansas Metropolitan Area': 'KS',
    'NC Area': 'NC',
    'AR Area': 'AR',
    'AZ Area': 'AZ'
}

#We made another keyword dictionary for all of the regular entries to be remapped to the state abbreviation.
state_abbrev=state_abbreviations = {
    'New Jersey': 'NJ',
    'Colorado': 'CO',
    'Ohio': 'OH',
    'New York': 'NY',
    'Iowa': 'IA',
    'North Carolina': 'NC',
    'California': 'CA',
    'Nebraska': 'NE',
    'Florida': 'FL',
    'Michigan': 'MI',
    'Missouri': 'MO',
    'Tennessee': 'TN',
    'Alaska': 'AK',
    'Rhode Island': 'RI',
    'Alabama': 'AL',
    'Georgia': 'GA',
    'Texas': 'TX',
    'Pennsylvania': 'PA',
    'Massachusetts': 'MA',
    'Arizona': 'AZ',
    'Virginia': 'VA',
    'Washington': 'WA',
    'Wisconsin': 'WI',
    'Hawaii': 'HI',
    'Louisiana': 'LA',
    'Utah': 'UT',
    'Indiana': 'IN',
    'Minnesota': 'MN',
    'Maryland': 'MD',
    'Kentucky': 'KY',
    'Oregon': 'OR',
    'New Mexico': 'NM',
    'Illinois': 'IL',
    'Montana': 'MT',
    'Oklahoma': 'OK',
    'District of Columbia': 'DC',
    'Mississippi': 'MS',
    'South Carolina': 'SC',
    'Kansas': 'KS',
    'Arkansas': 'AR',
    'Connecticut': 'CT',
    'Nevada': 'NV',
    'Idaho': 'ID',
    'New Hampshire': 'NH',
    'Wyoming': 'WY',
    'South Dakota': 'SD',
    'North Dakota': 'ND',
    'Delaware': 'DE',
    'West Virginia': 'WV',
    'Vermont': 'VT',
    'Maine': 'ME'
}

#Replace specific locations and non-abbreviated states with an abbreviated state name. A number of postings did not
#specidy the city or state, instead it just said "United States". We assumed that these are remote jobs and thus put
#the state as US.
filtered_postings_df['State']=filtered_postings_df['State'].str.strip().replace(specific_areas_abbrev)
filtered_postings_df['State']=filtered_postings_df['State'].str.strip().replace(state_abbrev)

In [15]:
#Checking that the replacements were made correctly
filtered_postings_df['State'].unique()

array(['GA', 'TX', 'PA', None, 'MI', 'NY', 'US', 'CT', 'FL', 'IL', 'NJ',
       'IA', 'VA', 'MA', 'CA', 'WA', 'CO', 'MD', 'NC', 'OK', 'WI', 'DE',
       'UT', 'OH', 'AZ', 'IN', 'OR', 'MN', 'MO', 'AL', 'NE', 'HI', 'KY',
       'KS', 'NM', 'TN', 'WV', 'ME', 'LA', 'DC', 'AR', 'SC', 'RI', 'AK',
       'NH', 'MT', 'ND', 'MS', 'NV', 'WY', 'SD', 'ID', 'VT'], dtype=object)

**Analysing the data**

**Graph 1:Plotting the most posted skill level in the data industry.**

The data shows that companies are mainly looking for Mid level experience from applicants

In [16]:
import plotly.express as px

experience_count = filtered_postings_df.groupby(['Experience']).size()
experience_count_df = experience_count.reset_index(name='count')
experience_count_df=experience_count_df.sort_values(by='count', ascending=True)
fig=px.bar(experience_count_df, x='Experience', y='count', color='Experience', title='Count of experience level required for jobs')
fig.show()

**Graph 2:
Plotting the experience level by state.**

California and Texas are by far the biggest employers, with both states having the most postings for mid-level jobs, followed by senior jobs. Texas takes the lead with Junior positions and managerial positions. This follows recent trends of people, especially in tech, moving to Texas for the lower cost of living and better taxes.

In [17]:
experience_by_state = filtered_postings_df.groupby(['State', 'Experience']).size().reset_index(name='count')
fig = px.histogram(experience_by_state, x='count', y='State', color='Experience', title='Experience Levels by State')
fig.show()

**Graph 3: Plotting the most poular job titles**

By using a filtered dataFrame looking at job titles that inclide the word Analyst. We found that the most posted jobs on LinkedIn are Business Analyst and Data Analst.

In [18]:
grouped_jobs = filtered_postings_df.groupby(['title']).size().reset_index(name='count')
grouped_jobs=grouped_jobs.nlargest(15,'count').sort_values(by='count', ascending=True)
fig=px.bar(grouped_jobs, x='title', y='count', color='title', title='Most popular jobs', labels={'count':'Number of jobs', 'title':'Job title'})
fig.show()

**Graph 4: Plotting the top 10 employers**

We found that The Job Network, Cogent Communications, and Dice are the companies that posted the most amount jobs on LinkedIn. The Job Network and Dice are both career marketplaces.

In [19]:

employers = filtered_postings_df.groupby(['company_name']).size().reset_index(name='count')
employers_top_10=employers.nlargest(15, 'count')
employers_top_10=employers_top_10.sort_values(by='count', ascending=True)
fig=px.bar(employers_top_10, x='company_name', y='count', color='company_name', title='Top 10 employers')
fig.update_yaxes(title_text='No. of posts')
fig.update_layout(showlegend=False)
fig.show()

**Graph 5: Plotting Top 15 Description words**

We found the words Analyst, Data, Senior, and Business are included the most times in LinkedIn job postings

In [20]:
from collections import Counter
import re

def remove_special_characters(text):
    return re.sub(r'[^A-Za-z\s]', '', text)

filtered_postings_df['cleaned_title'] = filtered_postings_df['title'].apply(remove_special_characters)
all_text = ' '.join(filtered_postings_df['cleaned_title'])
word_counts = Counter(all_text.split())

top_words = pd.DataFrame(word_counts.most_common(15), columns=['word','count'])
print(top_words)

fig = px.pie(top_words, values='count', names='word')
fig.show()


          word  count
0      Analyst   4736
1         Data   1181
2       Senior    879
3     Business    789
4    Financial    491
5     Engineer    362
6           Sr    289
7    Scientist    234
8   Operations    199
9          and    185
10     Systems    174
11    Security    172
12          II    157
13     Support    150
14      Remote    143


In [21]:
file = '/content/drive/MyDrive/Hackathon/jobs/job_skills.csv'
skills_df = pd.read_csv(file)
skills_df.head(5)

,job_id,skill_abr
0,3884428798,MRKT
1,3884428798,PR
2,3884428798,WRT
3,3887473071,SALE
4,3887465684,FIN


In [22]:
skill_count = skills_df.groupby(['skill_abr']).size()

**Graph 6: Plotting Job Skills over Number of Postings**

For the following graphs, we looked at data from all of the job postings and not only jobs relating to data or analytics.

We found the most wanted skills for jobs on LinkedIn were IT, Sales, and Management.


In [23]:
import plotly.express as px

skill_count_df = skill_count.reset_index(name='count')
skill_count_df=skill_count_df.sort_values(by='count', ascending=True)
fig = px.bar(skill_count_df, x='skill_abr', y='count',color='skill_abr', title='Skills v Job Count', text='count', labels={'skill_abr': 'Skills', 'count': 'Number of Jobs per Skill'})
fig.show()

In [24]:
file = ('/content/drive/MyDrive/Hackathon/jobs/benefits.csv')
benefits_df = pd.read_csv(file)
benefits_df.head(5)

,job_id,inferred,type
0,3887473071,0,Medical insurance
1,3887473071,0,Vision insurance
2,3887473071,0,Dental insurance
3,3887473071,0,401(k)
4,3887473071,0,Student loan assistance


In [26]:
ji_file = ('/content/drive/MyDrive/Hackathon/jobs/job_industries.csv')
in_file = ('/content/drive/MyDrive/Hackathon/mappings/industries.csv')
ji_df = pd.read_csv(ji_file)
in_df = pd.read_csv(in_file)

In [27]:
merged_df1 = pd.merge(ji_df, in_df, on='industry_id')

final_merged_df = pd.merge(merged_df1, skills_df, on='job_id')

final_merged_df.head(5)

,job_id,industry_id,industry_name,skill_abr
0,3884428798,82,Book and Periodical Publishing,MRKT
1,3884428798,82,Book and Periodical Publishing,PR
2,3884428798,82,Book and Periodical Publishing,WRT
3,3887467939,82,Book and Periodical Publishing,SALE
4,3887467939,82,Book and Periodical Publishing,ADVR


**Graph 7: Plotting the Number of Skills per Industry**

We merged two dataFrames - one including Skills per job posting and the other including Industries of each Company that is posting jobs. Again, this is looking at all the jobs data, so it is unsurprising that the healthcare industry is such a dominant employer in the USA with retail coming in a close second.

In [29]:
skills_industry = final_merged_df.groupby(['industry_name', 'skill_abr']).size().reset_index(name='count')
filtered_skills_industry = skills_industry[skills_industry['count'] >= 600]
filtered_skills_industry=filtered_skills_industry.sort_values(by='count', ascending=True)
fig = px.histogram(filtered_skills_industry, x='count', y='industry_name', color='skill_abr', title='Skills per Industry')
fig.show()

**Graph 8: Comparing the most popular types of job benefits**

We found the most given benefits from companies were 401(k), Medical Insurance, and Vision Insurance.

In [25]:
import plotly.express as px

benefit_count = benefits_df.groupby(['type']).size()
benefit_count_df = benefit_count.reset_index(name='count')
benefit_count_df=benefit_count_df.sort_values(by='count', ascending=True)
fig = px.bar(benefit_count_df, x='type', y='count',color='type', title='Benefit v Job Count', text='count',
                  labels= {"type": "Type of Benefit", "count": "Number of jobs with Benefit"})
fig.show()